***חלק א***

**סעיף א**

In [1]:
import pandas as pd

In [2]:
#התחברות לדרייב לשימוש בקבצים
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file_path='/content/drive/MyDrive/hadasim task/logs.txt.xlsx'

In [12]:
#מספר השורות בחלקים שאליהם נחלק את הקובץ
slice_size = 1000
start_row = 0

# יצירת סדרת נתונים ריקה לאיחוד השכיחויות של החלקים השונים
merged_frequencies = pd.Series(dtype=int)

# לולאה שתרוץ עד שנעבור על כל הקובץ
while True:
    try:
        #קריאה של חלק מתוך הקובץ
        slice = pd.read_excel(file_path, sheet_name="גיליון1", header=None,
                              skiprows=start_row, nrows=slice_size)

        # עצירה כשאין יותר שורות לקרוא
        if slice.empty:
            break

        # חילוץ קוד השגיאה מכל שורה
        errors_codes = slice.iloc[:, 0].str.split("Error:").str[1].str.strip()

        # ספירת שכיחויות של קודי שגיאה
        frequencies = errors_codes.value_counts()

        # איחוד שכיחויות מהחלק הנוכחי עם שאר השכיחויות
        merged_frequencies = merged_frequencies.add(frequencies, fill_value=0)

        # עדכון מיקום התחלה לקריאה הבאה
        start_row += slice_size

    except Exception as e:
        print("שגיאה בקריאה או עיבוד:", e)
        break

# הצגת התוצאה
N = 15
top_errors = merged_frequencies.sort_values(ascending=False).head(N)

# המרה ל-DataFrame והוספת כותרות בעברית
top_errors_df = pd.DataFrame({
    'קוד שגיאה': top_errors.index,
    'שכיחות': top_errors.values
})
# הדפסה
print(top_errors_df.to_string(index=False))


קוד שגיאה   שכיחות
 WARN_101 200098.0
  ERR_404 200094.0
  ERR_400 200069.0
 INFO_200 199931.0
  ERR_500 199808.0


# **סיבוכיות:**
**זמן:** O(n) עוברים על השורות אחת אחת בלי ביקה כפולה
(אם n שורות ו־ slice_size = k, אז יהיו בערך n/k קריאות לקובץ. כל אחת מהן בגודל קבוע, לכן הסיבוכיות הכוללת נשארת O(n).)

**מקום**: רק slice_size (שכאן בחרתי 1000) שורות נמצאות בזיכרון בכל רגע →

In [ ]:
N=15
sorted_frequencies = merged_frequencies.sort_values(ascending=False)
N_max_freq=sorted_frequencies.head(N)
print(N_max_freq)

**סעיף ב**

In [14]:
file_path='/content/drive/MyDrive/hadasim task/time_series.xlsx'

In [15]:
# קריאה של הקובץ
df = pd.read_excel(file_path, sheet_name="time_series" )  # הקובץ יקרא כגיליון שלם
print(df.head())

            timestamp value
0 2025-06-28 12:00:52  18.5
1 2025-06-01 04:17:23  46.3
2 2025-06-10 17:02:57    76
3 2025-06-23 05:23:22  56.4
4 2025-06-05 07:20:08  67.9


א.	כתיבת קטע קוד המבצע בדיקות לפני עיבוד הנתונים

In [16]:
def fix_data(df):
    # המרת תאריכים לא תקניים ל-NULL
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    # המרת הערכים בעמודת 'value' למספרים, והמרת ערכים לא תקניים ל-NULL
    df['value'] = pd.to_numeric(df['value'], errors='coerce')

    # הסרת שורות עם ערכים חסרים
    df = df.dropna()

    # הסרת כפילויות בתאריכים, כאשר שומרים את הרשומה הראשונה
    df = df.drop_duplicates(subset='timestamp', keep='first')

    return df

In [17]:
df = fix_data(df)

ב.	כתיבת קטע קוד המחשב את הערך הממוצע עבור כל שעה.

In [18]:
def compute_hourly_avg(df):
    # הוספת עמודה של שעה בלבד מתוך ה-Timestamp
    df['Hour'] = df['timestamp'].dt.floor('h')
    # קיבוץ לפי שעה וחישוב ממוצע
    hourly_avg = df.groupby('Hour')['value'].mean().reset_index()

    hourly_avg.rename(columns={'Hour': 'זמן התחלה', 'value': 'ממוצע'}, inplace=True)


    return hourly_avg

In [19]:
# הפעלת הפונקציה
hourly_avg = compute_hourly_avg(df)
# הדפסת התוצאה
print(hourly_avg)

              זמן התחלה      ממוצע
0   2025-06-01 00:00:00  49.824309
1   2025-06-01 01:00:00  50.564510
2   2025-06-01 02:00:00  49.478399
3   2025-06-01 03:00:00  50.264079
4   2025-06-01 04:00:00  48.939780
..                  ...        ...
715 2025-06-30 19:00:00  51.410565
716 2025-06-30 20:00:00  49.705429
717 2025-06-30 21:00:00  50.631281
718 2025-06-30 22:00:00  48.373709
719 2025-06-30 23:00:00  50.395344

[720 rows x 2 columns]


2.חילוק הקובץ לחלקים קטנים, חישוב עבור כל חלק ומיזוג של תוצאות החלקים

In [20]:
#נחלק לחלקים לפי ימים שונים
df['Date'] = df['timestamp'].dt.date

In [21]:
#פונקציה ליצירת קובץ csv עם הממוצעים
def hourly_avg_to_csv(df):
  # שמירת התוצאה בקובץ CSV
  df.to_csv('/content/drive/MyDrive/hadasim task/hourly_avg.csv', index=False)

In [24]:
#יצירת data frame ריק
#אליו נמזג את כל החלקים (הימים) השונים
merged_parts = pd.DataFrame()
# מעבר על כל יום בנפרד
for date, group in df.groupby('Date'):
    # חישוב ממוצע לשעה
    hourly_avg = hourly_avg = compute_hourly_avg(group)
    merged_parts = pd.concat([merged_parts, hourly_avg], ignore_index=True)
# הצגת התוצאה
print(merged_parts)
#hourly_avg_to_csv(merged_parts)

              זמן התחלה      ממוצע
0   2025-06-01 00:00:00  49.824309
1   2025-06-01 01:00:00  50.564510
2   2025-06-01 02:00:00  49.478399
3   2025-06-01 03:00:00  50.264079
4   2025-06-01 04:00:00  48.939780
..                  ...        ...
715 2025-06-30 19:00:00  51.410565
716 2025-06-30 20:00:00  49.705429
717 2025-06-30 21:00:00  50.631281
718 2025-06-30 22:00:00  48.373709
719 2025-06-30 23:00:00  50.395344

[720 rows x 2 columns]


3.התאמת החישוב למצב של זרימת נתונים בזמן אמת:

עבור עדכון הממוצעים השעתיים בזמן אמת:
בהנתן מבנה הנתונים ובו הממוצע הקיים עבור כל שעה:
נוסיף עמודה למבנה זה ובה ישמר מספר המקורות - מספר הנתונים ממנו חושב הממוצע הקיים.

כאשר נתון חדש מגיע, נעדכן את הממוצע השעתי כך:
נכפול את הממוצע הקודם במספר המקורות הקיים ,
נוסיף את ערך הנתון החדש.
נחלק את התוצאה במספר המקורות החדש (שהוא מספר המקורות הנוכחי + 1).
נעדכן את מספר המקורות על ידי הוספת 1.
נעדכן את הממוצע של אותה השעה בממוצע החדש שקבלנו


In [26]:
#נשנה את הפונקציה ששימשה לנו בסעיף הקודם כך שתחשב ותשמור גם את מספר הנתונים בדאטה המתקבלת

def hourly_avg_stream(df):
    # הוספת עמודה של שעה בלבד מתוך ה-Timestamp
    df['Hour'] = df['timestamp'].dt.floor('h')

    # קיבוץ לפי שעה, חישוב ממוצע וספירת הנתונים לכל שעה
    hourly_avg = df.groupby('Hour').agg({'value': 'mean', 'timestamp': 'count'}).reset_index()

    hourly_avg.rename(columns={'Hour': 'זמן התחלה', 'value': 'ממוצע', 'timestamp': 'מספר מקורות'}, inplace=True)

    return hourly_avg

In [27]:
#ניצור פונקציה המקבלת את הערכ\ים החדש\ים ואת המבנה נתונים שבו הממוצעים לפי שעות, מעדכנת אותו ומחזירה אותו מעודכן

def hourly_update(df,new_data):
    # המרת ה-timestamp לשעה עגולה
    new_data['timestamp'] = pd.to_datetime(new_data['timestamp']).dt.floor('h')

    #ניקח בחשבון שיכול להכנס בכל פעם נתון אחד או מספר נתונים ושהם נכנסים בצורת data frame
    # מעבר על כל השורות ב-new_data
    for _, row in new_data.iterrows():
        new_hour = row['timestamp']
        new_value = row['value']

        # אם כבר יש נתון עבור השעה והתאריך האלה
        if new_hour in df['זמן התחלה'].values:
            # מציאת השורה המתאימה לאותה השעה
            old_avg = df.loc[df['זמן התחלה'] == new_hour, 'ממוצע'].values[0]
            old_count = df.loc[df['זמן התחלה'] == new_hour, 'מספר מקורות'].values[0]

            # עדכון הממוצע והספירה
            new_avg = (old_avg * old_count + new_value) / (old_count + 1)
            df.loc[df['זמן התחלה'] == new_hour, 'ממוצע'] = new_avg
            df.loc[df['זמן התחלה'] == new_hour, 'מספר מקורות'] = old_count + 1
        else:
            # אם אין נתון עבור השעה, פשוט הוסף אותו כנתון חדש
            df = pd.concat([df, pd.DataFrame({'זמן התחלה': [new_hour], 'ממוצע': [new_value], 'מספר מקורות': [1]})], ignore_index=True)

    return df

In [29]:
# נתון חדש (דוגמת זרימה בזמן אמת)
new_data = pd.DataFrame({
    'timestamp': ['2025-06-01 00:00:00', '2025-06-01 01:00:00'],
    'value': [20.0, 30.0]
})

#נריץ בדיקה גם עבור הנתונים החדשים שהם אכן בסדר
new_data = fix_data(new_data)

#נניח שאנחנו מקבלים את df מהסעיפים הקודמים
#הפעלת הפונקציה המחשבת את הערך הממוצע עבור כל שעה וסופרת את הנתונים
hourly_avg=hourly_avg_stream(df)
#print(hourly_avg.head())
#עדכון הממוצעים הכולל את הדאטה החדשה
hourly_avg=hourly_update(hourly_avg,new_data)

#נרצה לקבל רק את נתוני הממוצעים לפי שעה ובלי מספר המקורות..
hourly_avg_neto = hourly_avg.drop(columns=['מספר מקורות'])
print(hourly_avg_neto)
#hourly_avg_to_csv(hourly_avg_neto)

              זמן התחלה      ממוצע
0   2025-06-01 00:00:00  49.795905
1   2025-06-01 01:00:00  50.544962
2   2025-06-01 02:00:00  49.478399
3   2025-06-01 03:00:00  50.264079
4   2025-06-01 04:00:00  48.939780
..                  ...        ...
715 2025-06-30 19:00:00  51.410565
716 2025-06-30 20:00:00  49.705429
717 2025-06-30 21:00:00  50.631281
718 2025-06-30 22:00:00  48.373709
719 2025-06-30 23:00:00  50.395344

[720 rows x 2 columns]


4.התאמת הקוד לפורמט PARQUET

In [36]:
file_path='/content/drive/MyDrive/hadasim task/time_series (4).parquet'

In [41]:
# קריאה של הקובץ
df = pd.read_parquet(file_path )
#print(df)

נשתמש בסעיף 3 כך שיתאים גם לאפשרות שיש זרימה בזמן אמת, מה שגם מתאים לנו מבחינת העובדה שנתונים כבר הממוצע ו\מספר המקורות

In [38]:
#נשמור את העמודות נחוצות לנו
#נשנה את שם עמודת הממוצע כדי שיתאים לפונקציית בדיקת הנתונים
df = df[['timestamp', 'mean_value', 'count']].rename(columns={'mean_value': 'value'})
#print(df.head())

#נריץ בדיקה  עבור הנתונים שהם אכן בסדר
df = fix_data(df)

#כעת נשנה את שמות העמודות כך שיתאימו לפונקציות ושיודפסו כמו שרצוי
df.rename(columns={'timestamp': 'זמן התחלה', 'value': 'ממוצע', 'count': 'מספר מקורות'}, inplace=True)
#print(df.head())


In [ ]:
# נתון חדש (דוגמת זרימה בזמן אמת)
#new_data = pd.DataFrame({
#    'timestamp': ['2025-06-01 00:00:00', '2025-06-01 01:00:00'],
#    'value': [20.0, 30.0]
#})

#נריץ בדיקה גם עבור הנתונים החדשים שהם אכן בסדר
#new_data = fix_data(new_data)

#נניח שאנחנו מקבלים את df מהסעיפים הקודמים
#הפעלת הפונקציה המחשבת את הערך הממוצע עבור כל שעה וסופרת את הנתונים
#hourly_avg=hourly_avg_stream(df)
#print(hourly_avg.head())
#עדכון הממוצעים הכולל את הדאטה החדשה
#hourly_avg=hourly_update(hourly_avg,new_data)


In [39]:
#נרצה לקבל רק את נתוני הממוצעים לפי שעה ובלי מספר המקורות..
hourly_avg_neto = df.drop(columns=['מספר מקורות'])
print(hourly_avg_neto)
hourly_avg_to_csv(hourly_avg_neto)

              זמן התחלה      ממוצע
0   2025-06-01 00:00:00  49.985786
1   2025-06-01 01:00:00  50.492752
2   2025-06-01 02:00:00  49.627839
3   2025-06-01 03:00:00  50.092934
4   2025-06-01 04:00:00  49.514965
..                  ...        ...
691 2025-06-29 19:00:00  50.244835
692 2025-06-29 20:00:00  49.963043
693 2025-06-29 21:00:00  49.703735
694 2025-06-29 22:00:00  49.816577
695 2025-06-29 23:00:00  49.782904

[696 rows x 2 columns]


 היתרונות באחסון המידע בפורמט הנתון(PARQUET):

 -כמו שניתן לראות גודל קובץ הPARQUET הוא מזערי לעומת גודל קובץ הXLSX
מה שאומר שיש לו יכולת דחיסה הרבה יותר טובה
 -כמו שעוד ניתן לראות קריאת הנתונים מהקובץ נעשית במהירות גדולה יותר מאשר קריאת הנתונים מקובץ האקסל

 באופן כללי עוד מיתרונות קובץ PARQUET:
 הנתונים נשמרים לפי סוגם: טקסט, מספרים וכו
 הנתונים נשמרים בעמודות (שלא לדוגמה כמו באקסל ששומר בשורות מה שגורם לזמן רב יותר הנדרש לקריאת הנתונים)

לסיכום:
 פורמט PARQUET טוב לחיסכון במקום, לשליפה מהירה של נתונים ושמירה על דיוקם. ובעצם מתאים לעבודה עם מאגרי נתונים גדולים.